In [1]:
import tensorflow as tf

## variable_scope变量作用域机制在Tensorflow中主要由两部分构成：

__v = tf.get_variable(name,shape,dtype,initializer)__

__tf.variable_scope(<scope_name>)__

+ 当tf.get_variable_scope().reuse == False时，
variable_scope作用域只能用来创建新变量



+ 当 tf.get_variable_scope().reuse == True 时，作用域可以共享变量

In [3]:
with tf.variable_scope('foo'):
    v = tf.get_variable('v',[1])
    v2= tf.get_variable('v',[1])
assert v.name == 'foo/v:0'

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-2-51f1d6c205b2>", line 2, in <module>
    v = tf.get_variable('v',[1])
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [2]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])

In [3]:
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
    print(v == v1) # 输出为 `True`代表v, v1 是相同的变量

True


--------------------------------------
### 获取变量作用域
可以直接通过 tf.variable_scope()来获取变量作用域:

In [2]:
with tf.variable_scope('foo') as foo_scope:
    v = tf.get_variable('v',[1])
with tf.variable_scope(foo_scope):
    w = tf.get_variable('w',[1])

In [3]:
with tf.variable_scope('foo') as foo_scope:
    assert foo_scope.name == 'foo'

In [4]:
with tf.variable_scope('bar'):
    with tf.variable_scope('baz') as other_scope:
        assert other_scope.name =='bar/baz'

----------------
### 变量作用域的初始化

变量作用域可以默认携带一个初始化器，在这个作用域中的子作用域或变量都可以继承或
者重写父作用域初始化器中的值。方法如下:

In [10]:
with tf.variable_scope('foo',initializer=tf.constant_initializer(0.4),reuse=True):
    v = tf.get_variable('v',[1])
    assert v.eval() == 0.4
    w = tf.get_variable('w',[1],initializer=tf.constant_initializer(0.3))
    assert w.eval() == 0.3
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
        assert v.eval() == 0.4 # 继承默认的初始化器
    with tf.variable_scope("baz", initializer=tf.constant_initializer(0.2)):
        v = tf.get_variable("v", [1])
        assert v.eval() == 0.2 # 重写父作用域的初始化器的值   

FailedPreconditionError: Attempting to use uninitialized value foo/v
	 [[Node: _send_foo/v_0 = _Send[T=DT_FLOAT, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=-93072411095750601, tensor_name="foo/v:0", _device="/job:localhost/replica:0/task:0/cpu:0"](foo/v)]]

## name_scope

TensorFlow 中常常会有数以千计的节点，在可视化的过程中很难一下子展示出来，因此 用 name_scope 为变量划分范围，在可视化中，这表示在计算图中的一个层级。name_scope 会影响 op_name，不会影响用 get_variable()创建的变量，而会影响通过 Variable()创建的变 量。

In [3]:
with tf.variable_scope('foo') as foo:
    with tf.name_scope('bar'):
        v = tf.get_variable('v',[1])
        b = tf.Variable(tf.zeros([1]),name='b')
        x = 1. + v
assert v.name == 'foo/v:0'
assert b.name == 'foo/bar/b:0'
assert x.op.name == 'foo/bar/add'

In [13]:
with tf.variable_scope('foo',initializer=tf.constant_initializer(0.4),reuse=True):
    v = tf.get_variable('v',[1])
    w = tf.get_variable('w',[1])

ValueError: Variable foo/w does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

In [4]:
with tf.variable_scope("one",reuse=True):
    a = tf.get_variable("v", [1]) #a.name == "one/v:0"
    c = tf.get_variable("v", [1])